In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OrdinalEncoder, KBinsDiscretizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
survey_df = pd.read_csv("Surveydata_train.csv")
survey_df_test = pd.read_csv("Surveydata_test.csv")
display(survey_df.head())
display(survey_df_test.head())

In [ ]:
display(survey_df.info())
display(survey_df_test.info())

In [ ]:
display(survey_df.iloc[:,2:17].describe(include = 'all'))
display(survey_df_test.iloc[:,1:16].describe(include = 'all'))

In [ ]:
display(survey_df.isna().sum())
display(survey_df_test.isna().sum())

In [ ]:
travel_df = pd.read_csv("Traveldata_train.csv")
display(travel_df.head())
travel_df_test = pd.read_csv("Traveldata_test.csv")
display(travel_df_test.head())

In [ ]:
display(travel_df.info())
display(travel_df_test.info())

In [ ]:
display(travel_df.iloc[:,1:9].describe(include ='all'))
display(travel_df_test.iloc[:,1:9].describe(include ='all'))

In [ ]:
display(travel_df.isna().sum())
display(travel_df_test.isna().sum())

In [ ]:
# Merge the two datasets

merged_df = pd.merge(survey_df, travel_df, on= 'ID')
display(merged_df.head())

merged_df_test = pd.merge(survey_df_test, travel_df_test, on= 'ID')
display(merged_df_test.head())

In [ ]:
display(merged_df.info())
display(merged_df_test.info())

In [ ]:
display(merged_df.isna().sum())
display(merged_df_test.isna().sum())

**Conclusion:** Categorical columns CustomerType, TypeTravel, Arrival_time_convenient, Catering & Onboard_service will need ML-based imputation.

### Imputing Cleanliness

In [ ]:
merged_df[merged_df['Cleanliness'].isna()]

**Conclusion**: Missing Cleanliness values are predominantly for fairly poor experiences: 
 - Overall_Experience 1
 - Seat_comfort extremely poor
 - Seat_Class Green Car
 - Arrival_time_convenient poor
 - Catering extremely poor
 - Platform_location manageable
 - Onboard_entertainment extremely poor
 - TypeTravel Personal Travel
 - Travel_Class Eco


In [ ]:
similar_cleanliness = merged_df.query("Overall_Experience == 1 & Seat_comfort == 'extremely poor' & Seat_Class == 'Green Car' & Arrival_time_convenient == 'poor' & Catering == 'extremely poor' & Platform_location == 'manageable' & Onboard_entertainment == 'extremely poor' & TypeTravel == 'Personal Travel' & Travel_Class == 'Eco'")

In [ ]:
similar_cleanliness['Cleanliness'].value_counts()

In [ ]:
similar_cleanliness.dropna(subset=['Cleanliness'])[['Online_support', 'Onboardwifi_service', 'Age', 'Cleanliness']]

In [ ]:
tech_cleanliness = merged_df[['Online_support', 'Onboardwifi_service', 'Cleanliness']].replace(['excellent', 'good', 'acceptable', 'need improvement', 'poor', 'extremely poor'], [5, 4, 3, 2, 1, 0])
sns.barplot(data=tech_cleanliness, x='Online_support', y='Cleanliness', hue='Onboardwifi_service')
plt.show()

In [ ]:
merged_df[merged_df['Cleanliness'].isna()][['Online_support', 'Onboardwifi_service', 'Cleanliness']]

In [ ]:
gender_age_cleanliness = merged_df[['Age', 'Gender', 'Cleanliness']].replace(['excellent', 'good', 'acceptable', 'need improvement', 'poor', 'extremely poor'], [5, 4, 3, 2, 1, 0])
gender_age_cleanliness['Age'] = pd.cut(merged_df['Age'], 5, labels = ['25', '35', '45', '60', '80'])
sns.barplot(data=gender_age_cleanliness, x='Age', y='Cleanliness', hue='Gender')
plt.show()

**Conclusion:** 
From exploring the dataset, it seems that:
 - Cleanliness rating is independent of Gender and Age.
 - Cleanliness depends more on tech services, like wifi and online support.
 - Similar reviews to the ones that need to be imputed are Acceptable/Good in a 1:1 ratio.
 - Based on the bar chart comparing Cleanliness values per wifi and online support rating, I'd suggest imputing 'good' for all but one missing value (the one is a combination of 'need improvement' for online support and 'acceptable' for wifi).

### Imputing Online boarding

In [ ]:
merged_df[merged_df['Online_boarding'].isna()]

In [ ]:
similar_boarding = merged_df.query("Overall_Experience == 1 & Seat_comfort == 'extremely poor' & Seat_Class == 'Green Car' & Arrival_time_convenient == 'poor' & Catering == 'extremely poor' & Platform_location == 'manageable' & Onboard_entertainment == 'extremely poor' & TypeTravel == 'Personal Travel' & Travel_Class == 'Eco'")

In [ ]:
similar_boarding['Online_boarding'].value_counts()

In [ ]:
tech_boarding = merged_df[['Online_support', 'Onboardwifi_service', 'Online_boarding']].replace(['excellent', 'good', 'acceptable', 'need improvement', 'poor', 'extremely poor'], [5, 4, 3, 2, 1, 0])
sns.barplot(data=tech_boarding, x='Online_support', y='Online_boarding', hue='Onboardwifi_service')
plt.show()

In [ ]:
merged_df[merged_df['Cleanliness'].isna()][['Online_support', 'Onboardwifi_service', 'Online_boarding']]

In [ ]:
gender_age_boarding = merged_df[['Age', 'Gender', 'Online_boarding']].replace(['excellent', 'good', 'acceptable', 'need improvement', 'poor', 'extremely poor'], [5, 4, 3, 2, 1, 0])
gender_age_boarding['Age'] = pd.cut(merged_df['Age'], 5, labels = ['25', '35', '45', '60', '80'])
sns.barplot(data=gender_age_boarding, x='Age', y='Online_boarding', hue='Gender')
plt.show()

**Conclusions**:
 - Online boarding depends highly on other tech services.
 - Doesn't depend that much on age or gender.
 - I'd suggest an imputation based on the bar chart of other tech services: 'excellent' for both excellent rows, 'poor' for both poor rows, 'good' for the good row and 'acceptable' for the remaining row.

### Imputing Onboard entertainment

In [ ]:
merged_df[merged_df['Onboard_entertainment'].isna()]

In [ ]:
fun_transformed = merged_df.replace(['excellent', 'good', 'acceptable', 'need improvement', 'poor', 'extremely poor'], [5, 4, 3, 2, 1, 0])

In [ ]:
sns.clustermap(fun_transformed.corr(), cmap="rocket_r")
plt.show()

In [ ]:
corr_fun = fun_transformed[['Online_support', 'Seat_comfort', 'Onboard_entertainment']].replace(['excellent', 'good', 'acceptable', 'need improvement', 'poor', 'extremely poor'], [5, 4, 3, 2, 1, 0])
sns.barplot(data=corr_fun, x='Online_support', y='Onboard_entertainment', hue='Seat_comfort')
plt.show()

In [ ]:
fun_df = merged_df[['Overall_Experience', 'Online_support', 'Seat_comfort']].dropna()
fun_df['Onboard_entertainment'] = merged_df['Onboard_entertainment']

In [ ]:
class CustomOrdinalEncoder:
    def __init__(self, categories):
        self.categories = categories
        self.cat_to_int = {}
        self.int_to_cat = {}
        for i, cat in enumerate(self.categories):
            self.cat_to_int[cat] = i
            self.int_to_cat[i] = cat

    def transform(self, data):
        return np.array([self.cat_to_int[cat] if cat in self.cat_to_int else np.nan for cat in data])

    def inverse_transform(self, data):
        return np.array([self.int_to_cat[int(cat)] for cat in data])

def encode_ordinal_columns(df, ordinal_columns, n_classes):
    encoders = {}
    encoded_df = df.copy()
    for col in ordinal_columns:
        unique_values = sorted(df[col].dropna().unique())
        categories = unique_values + [f"extra_class_{i}" for i in range(n_classes - len(unique_values))]
        encoder = CustomOrdinalEncoder(categories)
        encoded_df[col] = encoder.transform(df[col])
        encoders[col] = encoder
    return encoded_df, encoders

def impute_missing_ordinal_records(df, ordinal_columns, n_neighbors=3, n_classes=5):
    encoded_df, encoders = encode_ordinal_columns(df, ordinal_columns, n_classes)
    
    imputer = KNNImputer(n_neighbors=n_neighbors, weights='uniform')
    imputed_array = imputer.fit_transform(encoded_df)

    imputed_df = pd.DataFrame(imputed_array, columns=df.columns)
    imputed_df[ordinal_columns] = np.round(imputed_df[ordinal_columns])

    for col in ordinal_columns:
        imputed_df[col] = encoders[col].inverse_transform(imputed_df[col])

    return imputed_df

In [ ]:
fun_imputed = impute_missing_ordinal_records(fun_df, list(fun_df.columns), n_classes=5)
fun_imputed

In [ ]:
fun_imputed[merged_df['Onboard_entertainment'].isna()]

**Conclusions**:
 - Onboard entertainment is correlated with Overall experience, Online Support and Seat Comfort
 - Imputation can use all three parameters to fill in the missing values
 - An imputer of categorical ordinal data was created

### Imputing Platform location

In [ ]:
merged_df['Platform_location'].value_counts()

In [ ]:
merged_df[merged_df['Platform_location'].isna()]

In [ ]:
platform_transformed = (
    merged_df
    .replace(['excellent', 'good', 'acceptable', 'need improvement', 'poor', 'extremely poor'], [5, 4, 3, 2, 1, 0])
    .replace(['very convinient', 'Convinient', 'manageable', 'need improvement', 'Inconvinient', 'very inconvinient'], [5, 4, 3, 2, 1, 0])
)

In [ ]:
sns.clustermap(platform_transformed.corr(), cmap="rocket_r")
plt.show()

In [ ]:
sns.barplot(data=platform_transformed, x='Arrival_time_convenient', y='Platform_location', hue='Catering')
plt.show()

In [ ]:
platform_df = merged_df[['Arrival_time_convenient', 'Catering']].dropna()
platform_df['Platform_location'] = merged_df['Platform_location']

In [ ]:
platform_imputed = impute_missing_ordinal_records(platform_df, list(platform_df.columns), n_classes=5)
platform_imputed

**Conclusions:**
 - Platform location is correlated with Arrival time convenience and Catering
 - Imputer was used to impute missing data